## Setup

### Google Drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


### Dependencies

In [ ]:
!pip3 -q install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 torchtext==0.10.1 -f https://download.pytorch.org/whl/torch_stable.html

     |█████████████                   | 834.1 MB 14.9 MB/s eta 0:01:22tcmalloc: large alloc 1147494400 bytes == 0x3a8ae000 @  0x7f412c5f9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████▌               | 1055.7 MB 1.2 MB/s eta 0:13:41tcmalloc: large alloc 1434370048 bytes == 0x7ef04000 @  0x7f412c5f9615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████           | 1336.2 MB 1.5 MB/s eta 0:07:44tcmalloc: large alloc 1792966656 bytes == 0x3d36000 @  0x7f412c5f9615 0x592b76 0x4df71e 0x59afff 0x515655 0x

In [ ]:
# !pip install torchsummary==1.5.1

In [ ]:
!pip -qq install librosa
!pip -qq install sklearn

In [ ]:
# import torchsummary

In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.12.1+cu113
11.3


In [ ]:
torch.backends.cudnn.enabled

True

In [ ]:
torch.cuda.is_available()

True

In [ ]:
import numpy as np
import pandas as pd
import os
import torchaudio

### Device

In [ ]:
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"Using {device}")

Using cuda


## Dataset

### Customization

In [ ]:
from torch.utils.data import Dataset

class AudioDataset(Dataset):

  def __init__(self,
        annotations_file,
        audio_dir,
        transformation,
        target_sample_rate,
        num_samples,
        device):
    '''
    Arguments:
      annotations: path to csv file
      audio_dir: path to audio dir
      transformation: any audio tranform (Mel-spec, MFCC etc) provided by torch audio
               (function pointer in fact, callable)
      target_sample_rate: as named
    '''
    Dataset.__init__(self)
    self.annotations = pd.read_csv(annotations_file)
    print(self.annotations)
    self.audio_dir = audio_dir
    self.device = device
    # print(self.device)
    self.transformation = transformation.to(self.device)
    self.target_sample_rate = target_sample_rate
    self.num_samples = num_samples
  
  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    '''
    loading the waveform sample specified by the index
    
    Returns:
      signal: <transformed> waveform
      label: text in this case
    '''
    audio_sample_path = self._get_audio_sample_path(index)
    label = self._get_audio_sample_label(index)

    signal, sr = torchaudio.load(audio_sample_path)
    signal = signal.to(self.device)
    # signal = signal.cuda()
    # Note: signal has shape (num_channels, num_samples) -> (2, 16000) -> (1, 16000)

    signal = self._resample_if_necessary(signal, sr)  # resampling
    signal = self._mix_down_if_necessary(signal)    # reduce to mono-channel (aggregate over 0st dim)
    signal = self._cut_if_necessary(signal)
    signal = self._right_pad_if_necessary(signal)   # right padding (when have less than expected)
    signal = self.transformation(signal)       # apply tranformation

    return signal, label

  def _get_audio_sample_path(self, index):
    '''
    Join <dir name> & <.wav filename in df anno>
    retrieve complete index-th audio sample url

    Returns:
      path: complete i-th audio sample url
    '''

    file_name = f'{self.annotations.iloc[index, 0]}.wav' # 0st col: 'UID'
    path = os.path.join(self.audio_dir, file_name)
    return path
    
  def _get_audio_sample_label(self, index):
    # return self.annotations.iloc[index, 3] # 1st col: 'UTT' 4th: 'EMOTION'

    label1 = self.annotations.iloc[index, 3]
    # label2 = torch.from_numpy(self.annotations.iloc[index, 4:7].to_numpy().astype(np.float32))

    # return {'label1': label1, 'label2': label2}
    return {'label1': label1}

  def _cut_if_necessary(self, signal):
    if signal.shape[1] > self.num_samples:
      signal = signal[:, :self.num_samples]
    return signal

  def _right_pad_if_necessary(self, signal):
    length_signal = signal.shape[1]
    if length_signal < self.num_samples:
      num_missing_samples = self.num_samples - length_signal
      last_dim_padding = (0, num_missing_samples) #(1, 2) 1 left 2 right
      # [1, 1, 1] -> [0, 1, 1, 1, 0, 0]
      signal = torch.nn.functional.pad(signal, last_dim_padding)
    return signal

  def _resample_if_necessary(self, signal, sr):
    if sr != self.target_sample_rate:
      if self.device == "cpu":
        resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate)
      elif self.device == 'cuda':
        resampler = torchaudio.transforms.Resample(sr, self.target_sample_rate).cuda()
      else:
        raise Exception("Device type error!")
      signal = resampler(signal)
    return signal

  def _mix_down_if_necessary(self, signal):
    if signal.shape[0] > 1: # i.e. (2, 1000)
      signal = torch.mean(signal, dim=0, keepdim=True)
    return signal

### Instantiation

- Dataset
- Data loader

In [ ]:
ANNOTATIONS_FILE = "/content/drive/MyDrive/Dissertation/Datasets/cleaned_1.0/better_train_less.csv"
AUDIO_DIR = "/content/drive/MyDrive/Dissertation/Speech"

BATCH_SIZE = 64
# BATCH_SIZE = 32

EPOCHS = 10
LEARNING_RATE = 0.001

# SAMPLE_RATE = 16000
# NUM_SAMPLES = 220500 * 3

SAMPLE_RATE = 8000
NUM_SAMPLES = 8000 * 2
# NUM_SAMPLES = 16000 * 3

In [ ]:
from torch.utils.data import DataLoader

def create_data_loader(train_data, batch_size):
  train_dataloader = DataLoader(train_data, batch_size=batch_size)
  return train_dataloader

# instantiating our dataset object and create data loader
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
  sample_rate=SAMPLE_RATE,
  n_fft=1024,
  hop_length=512,
  n_mels=64
)

iemocap = AudioDataset(
      ANNOTATIONS_FILE,
      AUDIO_DIR,
      mel_spectrogram,
      SAMPLE_RATE,
      NUM_SAMPLES,
      device
      )

train_dataloader = create_data_loader(iemocap, BATCH_SIZE)

                         UID  \
0        Ses03M_impro04_M041   
1        Ses01M_impro06_M011   
2        Ses02M_impro08_M018   
3        Ses03M_impro03_M011   
4        Ses05F_impro03_F040   
...                      ...   
3125     Ses05M_impro01_F021   
3126  Ses04F_script03_2_F030   
3127     Ses05M_impro08_M024   
3128  Ses03M_script01_2_M004   
3129     Ses05F_impro06_F005   

                                                    UTT EMOTIONS  EMOTION  \
0     Like, wow you're really smart we're going to g...        3        3   
1     No, I mean it just came on really quick, you k...        2        2   
2                                                 Okay.        0        0   
3     Ah...Anyways, so she took the ring and she goe...        3        3   
4                          There's still time for that.        3        3   
...                                                 ...      ...      ...   
3125  I'm gonna- I'm gonna bring security over and h...        1        1   

In [ ]:
# import IPython.display as ipd

# print(f'<{len(iemocap)}> samples confirmed in the dataset.')

# print(iemocap[1])

# sample_signal, sample_label = iemocap[1]

# print(f'Sample Audio: {sample_signal.shape}')
# print(f'Sample Label: {sample_label}')


# # ipd.Audio(data=np.asarray(iemocap[1][0]), autoplay=True, rate=16000)
# ipd.Audio(data=np.asarray(sample_signal), autoplay=True, rate=16000)

## Tensor Split

In [ ]:
a = torch.arange(40).reshape(1,1,5,8)
a

tensor([[[[ 0,  1,  2,  3,  4,  5,  6,  7],
          [ 8,  9, 10, 11, 12, 13, 14, 15],
          [16, 17, 18, 19, 20, 21, 22, 23],
          [24, 25, 26, 27, 28, 29, 30, 31],
          [32, 33, 34, 35, 36, 37, 38, 39]]]])

In [ ]:
a.shape[-1]

8

In [ ]:
def slicing(tensor, r=4):
  len_time = a.shape[-1]
  assert len_time % r == 0, f'not dividable by r of {r}'
  len_small = len_time // r
  return [
      a[:,:,:, i:i+2] for i in range(0, len_time, len_small)
  ]

In [ ]:
slicing(a)

[tensor([[[[ 0,  1],
           [ 8,  9],
           [16, 17],
           [24, 25],
           [32, 33]]]]), tensor([[[[ 2,  3],
           [10, 11],
           [18, 19],
           [26, 27],
           [34, 35]]]]), tensor([[[[ 4,  5],
           [12, 13],
           [20, 21],
           [28, 29],
           [36, 37]]]]), tensor([[[[ 6,  7],
           [14, 15],
           [22, 23],
           [30, 31],
           [38, 39]]]])]

In [ ]:
len(slicing(a))

4

In [ ]:
torch.cat(slicing(a), -1)

tensor([[[[ 0,  1,  2,  3,  4,  5,  6,  7],
          [ 8,  9, 10, 11, 12, 13, 14, 15],
          [16, 17, 18, 19, 20, 21, 22, 23],
          [24, 25, 26, 27, 28, 29, 30, 31],
          [32, 33, 34, 35, 36, 37, 38, 39]]]])

## Architecture

In [ ]:
from torch import nn
from torchsummary import summary

- `__init__()`: the constructor defining layer structure of the network.

- `forward()`: instruct pytorch how to pass the information/data from one layer to next.

In [ ]:
import torch.nn.functional as F

In [ ]:
class CNNNetwork(nn.Module):

  def __init__(self):
    super().__init__()

    # sequential convolutions  / flatten / linear / softmax
    self.convSeq = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=16,  # 16 filters
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.Tanh(),
        # nn.Dropout2d(0.1),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(
            in_channels=16,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.Tanh(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(
            in_channels=32,
            out_channels=64,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.Tanh(),
        nn.MaxPool2d(kernel_size=2),
        nn.Conv2d(
            in_channels=64,
            out_channels=128,
            kernel_size=3,
            stride=1,
            padding=2
        ),
        nn.Tanh(),
        nn.MaxPool2d(kernel_size=2),
        nn.Flatten()

    )

    # flatten
    self.flatten = nn.Flatten()

    # linear/dense
    # self.linear = nn.Linear(128 * 5 * 4, 4)
    self.linear1 = nn.Sequential(
        # nn.Linear(128 * 1 * 8, 800),
        nn.Linear(64 * 5 * 4, 400),
        nn.Tanh(),
        # nn.Linear(54000, 4000),
        # nn.Tanh(),
        # nn.BatchNorm1d(1024),
        nn.Linear(400, 4)
    )

    # # VAD
    # self.linear2 = nn.Sequential(
    #     nn.Linear(64 * 5 * 4, 512),
    #     nn.Tanh(),
    #     nn.Linear(512, 300),
    #     nn.Tanh(),
    #     # nn.BatchNorm1d(1024),
    #     nn.Linear(300, 3)
    # )

    # softmax
    self.softmax = nn.Softmax(dim=1)
  
  # how to pass data through layers
  def forward(self, input_data):

    one = input_data[:,:,:, 0:input_data.shape[-1] // 4]
    two = input_data[:,:,:, input_data.shape[-1] // 4: input_data.shape[-1] // 4 * 2]
    thr = input_data[:,:,:, input_data.shape[-1] // 4 * 2:input_data.shape[-1] // 4 * 3]
    fou = input_data[:,:,:, input_data.shape[-1] // 4 * 3:input_data.shape[-1] // 4 * 4]

    one = self.convSeq(one)
    two = self.convSeq(two)
    thr = self.convSeq(thr)
    fou = self.convSeq(fou)

    # x = torch.cat((one,two,thr,fou), -1)
    # x = torch.cat((one,two,thr,fou), 0)

    x = torch.add(one, two)
    x = torch.add(x, thr)
    x = torch.add(x, fou)
    # x = one+two+thr+fou
    x = x * 0.25

    # x = F.pad(x, pad=(0, 0, 0, 1920 - x.shape[0]))
    # print(x.shape)
    x.to(device)

    x1 = self.linear1(x)
    # x2 = self.linear2(x)

    x1 = self.softmax(x1)
    # return x1, x2
    return x1

In [ ]:
# cnn_sample = CNNNetwork()
# summary(
#     cnn.cuda(),  # model
    
#      # shape of the spetrogram
#     (
#         1,  # number of channels
#         64,  # number of mel-banks
#         44  # time axis
#     )
#     )

## Training

### Train & Save

In [ ]:
import torch
import torchaudio
from torch import nn


# BATCH_SIZE = 128
# EPOCHS = 10
LEARNING_RATE = 0.001

# SAMPLE_RATE = 16000
# NUM_SAMPLES = 22050


def train_single_epoch(model, data_loader, loss_fn, optimiser, device, i):
  for input, target in data_loader:
    # input, target1, target2 = input.to(device), target['label1'].to(device), target['label2'].to(device)
    input, target1 = input.to(device), target['label1'].to(device)

    # calculate loss
    out1 = model(input)
    loss1 = loss_fn(out1, target1)
    # out1, out2 = model(input)
    # loss2 = nn.L1Loss()(out2, target2)

    # backpropagate error and update weights
    # loss = loss1 + loss2
    loss = loss1
    # writer.add_scalar("Loss/train", loss, i)
    optimiser.zero_grad()
    
    loss.backward()
    optimiser.step()

  print(f"loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimiser, device, epochs):
  for i in range(epochs):
      print(f"Epoch {i+1}")
      train_single_epoch(model, data_loader, loss_fn, optimiser, device, i)
      print("---------------------------")
  print("Finished training")

### Loss & Optimizer

In [ ]:
# construct model and assign it to device
cnn = CNNNetwork().to(device)
print(cnn)

# initialise loss funtion + optimiser
loss_fn = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(
                cnn.parameters(),
                lr=LEARNING_RATE
                )

CNNNetwork(
  (convSeq): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (7): Tanh()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (10): Tanh()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Sequential(
    (0): Linear(in_features=1280, out_features=400, bias=True)
    (1): Tanh()
    (2): Linear(in_features=400, out_features=4, bias=True)
  )


In [ ]:
# train model
train(cnn, train_dataloader, loss_fn, optimiser, device, EPOCHS)

Epoch 1
loss: 1.2133526802062988
---------------------------
Epoch 2
loss: 1.1466076374053955
---------------------------
Epoch 3
loss: 1.1346944570541382
---------------------------
Epoch 4
loss: 1.1251883506774902
---------------------------
Epoch 5
loss: 1.1225779056549072
---------------------------
Epoch 6
loss: 1.1245322227478027
---------------------------
Epoch 7
loss: 1.130554437637329
---------------------------
Epoch 8
loss: 1.135025143623352
---------------------------
Epoch 9
loss: 1.0455280542373657
---------------------------
Epoch 10
loss: 1.0457149744033813
---------------------------
Finished training


In [ ]:
# writer.flush()
# writer.close()

In [ ]:
# !kill 2108

In [ ]:
# tensorboard --logdir = content/logsdir
# %tensorboard --logdir content/drive/MyDrive/Dissertation/Models/logsdirs/logsdir1/

### Saving

In [ ]:
# output_path = "/content/drive/MyDrive/Dissertation/Models/SER/clean_1.0/CNN_series.pth"
output_path = "/content/drive/MyDrive/Dissertation/Models/SER/clean_1.0/CNN_series_no_vad_new.pth"

In [ ]:
# save model
torch.save(cnn.state_dict(), output_path)
print(f"Trained feed forward net saved at [{output_path}]")

Trained feed forward net saved at [/content/drive/MyDrive/Dissertation/Models/SER/clean_1.0/CNN_series_no_vad_new.pth]


## Evaluation

### Class Mapping

In [ ]:
class_mapping = ['neu','ang','sad','hap']
INDICE = [0, 1, 2, 3]

EMO_2_ID = dict(zip(class_mapping, INDICE))
ID_2_EMO = dict(zip(INDICE, class_mapping))

### Predictions

In [ ]:
def predict(model, input, target, class_mapping):
  model.eval()

  # no need for gradient during inference
  with torch.no_grad():
    # predictions, vad = model(input)
    # predictions, vads = model(input)
    predictions = model(input)
    # tensor (1, 4) -> [[0,1, 0.2, 0.6, 0.1]]

    predicted_index = predictions[0].argmax(0)
    # print(predicted_index)

    predicted = class_mapping[predicted_index]
    expected = class_mapping[target]
  return predicted, expected

### Load back

In [ ]:
cnn_trained = CNNNetwork()
state_dict = torch.load(output_path)
cnn_trained.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# ANNOTATIONS_FILE = "/content/drive/MyDrive/Dissertation/Datasets/cleaned_1.0/better_valid_less.csv"
ANNOTATIONS_FILE = "/content/drive/MyDrive/Dissertation/Datasets/cleaned_1.0/better_test_less.csv"
AUDIO_DIR = "/content/drive/MyDrive/Dissertation/Speech"

# instantiating our dataset object and create data loader
mel_spectrogram = torchaudio.transforms.MelSpectrogram(
  sample_rate=SAMPLE_RATE,
  n_fft=1024,
  hop_length=512,
  n_mels=64
)

iemocap_valid = AudioDataset(
      ANNOTATIONS_FILE,
      AUDIO_DIR,
      mel_spectrogram,
      SAMPLE_RATE,
      NUM_SAMPLES,
      'cpu'
      # device
      )

# valid_dataloader = create_data_loader(iemocap_valid, BATCH_SIZE)

                         UID  \
0        Ses04F_impro06_F005   
1        Ses04F_impro03_M021   
2     Ses04M_script02_2_F019   
3        Ses03M_impro06_M020   
4        Ses03F_impro04_F023   
...                      ...   
1039  Ses01F_script03_2_M020   
1040  Ses03M_script03_2_M021   
1041     Ses04M_impro07_M007   
1042  Ses05M_script03_2_M030   
1043  Ses01M_script01_3_F016   

                                                    UTT EMOTIONS  EMOTION  \
0     He was only twenty four and just so many thing...        2        2   
1                      I know but you just asked me to-        0        0   
2     I'd rather not remember some things.  I'd rath...        2        2   
3        and then something like this happens you know?        2        2   
4                                Yeah, I guess I could.      0,2        2   
...                                                 ...      ...      ...   
1039  Oh really.  It's a pity you didn't have any br...    0,1,2        1   

### Inference

In [ ]:
# [batch_size, num_channels, freq, time]
# input, target1, target2 = iemocap_valid[0][0], iemocap_valid[0][1]['label1'], iemocap_valid[0][1]['label2']
input, target1 = iemocap_valid[0][0], iemocap_valid[0][1]['label1']

# 0 is the index of insertion
input.unsqueeze_(0)

print(target1)

# input, target1 = input.cuda(), torch.tensor(target1).cuda()

predicted, expected = predict(cnn_trained, input, target1, class_mapping)

print(f'Predicted: {predicted}, expected: {expected}')
print(input.shape)
print(len(iemocap_valid))

2
Predicted: sad, expected: sad
torch.Size([1, 1, 64, 32])
1044


In [ ]:
from sklearn.metrics import confusion_matrix

def results(dataset, model):
  predictions = []
  expectations = []

  size = len(dataset)
  for i in range(size):
    X, Y = dataset[i][0], dataset[i][1]['label1']
    X.unsqueeze_(0)
    # X = X.cuda()
    # Y = torch.from_numpy(Y)
    # Y = Y.cuda()
    predicted, expected = predict(model, X, Y, class_mapping)
    predictions.append(predicted)
    expectations.append(expected)

  return expectations, predictions

y_true, y_predict = results(iemocap_valid, cnn_trained)

In [ ]:
len(y_true)

1044

In [ ]:
ID_2_EMO

{0: 'neu', 1: 'ang', 2: 'sad', 3: 'hap'}

In [ ]:
y_true = [EMO_2_ID[y] for y in y_true]
y_predict = [EMO_2_ID[y] for y in y_predict]

In [ ]:
from sklearn.metrics import classification_report
c = classification_report(y_true, y_predict, target_names=class_mapping, zero_division=1)
print(c)

              precision    recall  f1-score   support

         neu       0.49      0.60      0.54       323
         ang       0.58      0.66      0.62       219
         sad       0.60      0.70      0.65       230
         hap       0.43      0.20      0.27       272

    accuracy                           0.53      1044
   macro avg       0.52      0.54      0.52      1044
weighted avg       0.52      0.53      0.51      1044



In [ ]:
UA = 0
for i in range(len(y_true)):
  if y_true[i] == y_predict[i]:
    UA += 1
UA /= len(y_true)
UA


0.5316091954022989

In [ ]:
c_dict = classification_report(y_true, y_predict, target_names=class_mapping, zero_division=1, output_dict=True)
c_dict

{'neu': {'precision': 0.4911392405063291,
  'recall': 0.6006191950464397,
  'f1-score': 0.5403899721448469,
  'support': 323},
 'ang': {'precision': 0.5753968253968254,
  'recall': 0.6621004566210046,
  'f1-score': 0.6157112526539279,
  'support': 219},
 'sad': {'precision': 0.5955882352941176,
  'recall': 0.7043478260869566,
  'f1-score': 0.6454183266932271,
  'support': 230},
 'hap': {'precision': 0.432,
  'recall': 0.19852941176470587,
  'f1-score': 0.2720403022670025,
  'support': 272},
 'accuracy': 0.5316091954022989,
 'macro avg': {'precision': 0.523531075299318,
  'recall': 0.5413992223797767,
  'f1-score': 0.5183899634397511,
  'support': 1044},
 'weighted avg': {'precision': 0.5164168329148431,
  'recall': 0.5316091954022989,
  'f1-score': 0.509413699894696,
  'support': 1044}}

In [ ]:
df = pd.read_csv(ANNOTATIONS_FILE)
names = df.columns.to_list()
names[-4:]

['neu_m', 'ang_m', 'sad_m', 'hap_m']

In [ ]:
y_true_multi = df[['neu_m', 'ang_m', 'sad_m', 'hap_m']].to_numpy()
y_true_multi.shape

(1044, 4)

In [ ]:
y_true_multi

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       ...,
       [1, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0, 0, 1]])

In [ ]:
y_predict[:10]

[2, 1, 0, 2, 2, 1, 0, 0, 0, 0]

In [ ]:
# # y_predict_hot = pd.get_dummies(y_predict)
y_predict = np.array(y_predict)
# y_predict_hot = np.zeros(y_predict.size, y_predict.max() + 1)
# y_predict_hot[np.arange(y_predict.size), y_predict] = 1
# y_predict_hot

In [ ]:
# y_predict

In [ ]:
def one_hot(n, class_num, col_wise=True):
  a = np.eye(class_num)[n.reshape(-1)]
  return a.T if col_wise else a

In [ ]:
y_predict_hot = one_hot(y_predict, 4, False)
y_predict_hot

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [ ]:
c2 = classification_report(y_true_multi, y_predict_hot, target_names=class_mapping, zero_division=1)
print(c2)

              precision    recall  f1-score   support

         neu       0.60      0.52      0.56       454
         ang       0.61      0.61      0.61       251
         sad       0.66      0.67      0.67       269
         hap       0.45      0.20      0.28       279

   micro avg       0.60      0.50      0.54      1253
   macro avg       0.58      0.50      0.53      1253
weighted avg       0.58      0.50      0.53      1253
 samples avg       0.60      0.52      0.55      1253

